<a href="https://colab.research.google.com/github/sandeeptuluri/fresh1/blob/main/text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries.....

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Reading the CSV....

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/sandeeptuluri/fresh1/main/Data%20extract%20for%20Sandeep.csv')
df.head()

id  ...      topicName
0  26696  ...     Technology
1  26694  ...       Politics
2  26693  ...       Politics
3  26692  ...  North America
4  26690  ...       Politics

[5 rows x 4 columns]

In [ ]:
df.isnull().sum()

id           0
summary      0
topicID      0
topicName    0
dtype: int64

In [ ]:
df.topicName.nunique()

29

In [ ]:
df.describe()


id       topicID
count  12802.000000  12802.000000
mean   18746.503359      9.722231
std     4643.310327      6.670339
min    10858.000000      2.000000
25%    14614.250000      5.000000
50%    18707.500000      7.000000
75%    22829.750000     14.000000
max    26696.000000     33.000000

Data is very clean....and ready to use

In [ ]:
df['summary'] = df['summary'].apply(lambda summary: (summary).lower())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df['summary']
y = df['topicID']

Splitting the data for efficient use....

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
from tensorflow.keras.utils import to_categorical


In [ ]:
ytr = to_categorical(y_train)


In [ ]:
ytr.shape

(8961, 34)

In [ ]:
yte = to_categorical(y_test)

In [ ]:
yte.shape

(3841, 34)

In [ ]:
import tensorflow as tf


In [ ]:
import tensorflow_hub as hub

Importing the pretrained embedded layer from google....for better training the model...

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
#hub_layer(train)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

Preparing the model....

In [ ]:
model = Sequential()
model.add(hub_layer)
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(34, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Fitting the model....

In [ ]:
model.fit(X_train,ytr,epochs=10,validation_data=(X_test,yte),verbose=1)

Epoch 1/10
281/281 [==============================] - 151s 536ms/step - loss: 2.4312 - accuracy: 0.3269 - val_loss: 2.0635 - val_accuracy: 0.4064
Epoch 2/10
281/281 [==============================] - 150s 532ms/step - loss: 1.7555 - accuracy: 0.5009 - val_loss: 1.9309 - val_accuracy: 0.4400
Epoch 3/10
281/281 [==============================] - 149s 531ms/step - loss: 1.3058 - accuracy: 0.6074 - val_loss: 2.0825 - val_accuracy: 0.4212
Epoch 4/10
281/281 [==============================] - 150s 533ms/step - loss: 0.9298 - accuracy: 0.7068 - val_loss: 2.5124 - val_accuracy: 0.4033
Epoch 5/10
281/281 [==============================] - 149s 531ms/step - loss: 0.6402 - accuracy: 0.7944 - val_loss: 3.0267 - val_accuracy: 0.3895
Epoch 6/10
281/281 [==============================] - 149s 530ms/step - loss: 0.4510 - accuracy: 0.8614 - val_loss: 3.7702 - val_accuracy: 0.3601
Epoch 7/10
281/281 [==============================] - 150s 534ms/step - loss: 0.3054 - accuracy: 0.9092 - val_loss: 4.3043 -

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[['loss','val_loss']].plot()

In [ ]:
metrics[['accuracy','val_accuracy']].plot()

In [ ]:
model.metrics_names

['loss', 'accuracy']

evaluating the model.....

In [ ]:
model.evaluate(X_test,yte,verbose=0)

[5.554201602935791, 0.36032283306121826]

Predicting the model....

In [ ]:
predictions = model.predict(X_test)

In [ ]:
a = X_test[:1]

In [ ]:
a

5621    yeimo, a papuan activist was arrested and char...
Name: summary, dtype: object

In [ ]:
h = model.predict(a)

In [ ]:
h

array([[4.7434837e-11, 3.3666613e-12, 1.2492764e-05, 6.5413415e-01,
        2.3747793e-10, 1.1853002e-05, 4.8625605e-08, 1.7596791e-04,
        3.9511194e-04, 2.6246953e-01, 5.2314331e-06, 3.4277625e-03,
        6.3862205e-02, 3.0407091e-07, 5.3442864e-07, 1.4731835e-06,
        3.1076432e-08, 3.5671681e-11, 1.0722136e-04, 1.5667196e-05,
        2.8718852e-05, 6.6237987e-10, 3.9413466e-11, 8.6725099e-10,
        9.2194863e-10, 2.0507977e-11, 1.5351218e-02, 3.6657673e-07,
        6.4462704e-08, 1.7686824e-09, 5.7050227e-09, 7.2645867e-10,
        2.8889915e-10, 1.2575403e-08]], dtype=float32)

In [ ]:
np.argmax(h)

3

Here then number three means.... politics.... and it is perfectly predicted

Here, the model predicted the accurate result.... and the accuracy is also quiet good....

In [ ]:
!mkdir -p saved_model
model.save('saved_model/text_classification')

INFO:tensorflow:Assets written to: saved_model/text_classification/assets


INFO:tensorflow:Assets written to: saved_model/text_classification/assets


In [ ]:
ls

sample_data/  saved_model/


In [ ]:
ls saved_model/

text_classification/


In [ ]:
ls saved_model/text_classification

assets/  keras_metadata.pb  saved_model.pb  variables/


Follow from here......

Loading the model which has been already shared in the google colab.....

In [110]:
import tensorflow as tf

In [111]:
main = tf.keras.models.load_model('saved_model/text_classification/')

Testing the model by picking the sample summary from the test dataset.....

Here we are picking the 13 summary from the test dataset.....

In [112]:
k = X_test[12:13]

In [113]:
k

5099    german president frank-walter steinmeier annou...
Name: summary, dtype: object

In [127]:
u = main.predict(k)

In [128]:
np.argmax(u)

7

Here the model predicted correctly and result here "7" indicates that the summary related to "Global Health".......

Here we are giving the outside summary......

In [124]:
l = ["The number of people who lost their lives in Uttarakhand amid heavy rainfall rose to 47. The number of casualties in Kumaon alone has reached 42, DIG Nilesh Anand Bharne said. Of the 42 new casualties, 28 people were killed in Nainital. Notably, Union Home Minister Amit Shah will visit Uttarakhand on Wednesday to take stock of the situation."]

In [125]:
z = main.predict(l)

In [126]:
np.argmax(z)

9

Here "9" indicates that the news realted to "ASIA". and the result is correct.....

you can check the results down below in the groupby table.....

In [121]:
g = df.groupby(['topicID','topicName'])

In [101]:
g.first()

id                                            summary
topicID topicName                                                                 
2       Sports            26632  germany is set to play at the world cup 2022 i...
3       Politics          26694  the eu has announced a humanitarian aid packag...
4       Start-ups         26612  lego has had to “address gender bias and harmf...
5       Business          26662  adani ports, india’s largest port operator, sa...
6       Habitat           26676  the greek island of crete has been hit a 6.4-m...
7       Global health     26671  the uk's failure to do more to stop covid spre...
8       Europe            26678  cemetery outside berlin said it made a “terrib...
9       Asia              26688  leaders from g20 nations have stressed the imp...
10      North America     26692  governor sheinbaum has confirmed that a statue...
11      Africa            26655  the massive displacement in cabo delgado has p...
12      Middle East       26660  the complex, discovered in the central town of...
13      Technology        26696  expo 2020 dubai is finally open after an 8-mon...
14      Travel            26616  the dominican republic surpassed pre-pandemic ...
15      Gender Equality   26645  sally ride, the first american woman to travel...
16      Art and Culture   26611  pink has been named the most played female art...
18      Latin America     26618  biden has proclaimed oct. 11 as indigenous peo...
19      Social Movements  26420  south africa's biggest metalworkers union laun...
20      Local News        26675  govt. has decided to restore the scheduled dom...
21      Happy news        26687  the us plans to reopen its land borders with c...
24      Climate Crisis    26631  prince charles said he totally understands why...
25      Science           26654  the planet is tidally locked - one side of the...
26      Human Rights      26275  the us department of labor has published a rep...
27      Mental Health     26348  french psychiatry professionals are scrutinisi...
28      Biotech           17362  josé baselga, astrazeneca's executive vice pre...
29      LGBT              16595  sydney's annual mardi gras parade was held ins...
30      Finance           23601  the recent surge in consumer prices will be a ...
31      Education         25738  the former united states secretary of state hi...
32      Crypto Currency   21860  formula 1 landed a new sponsor tuesday in a de...
33      Human Stories     25928  in russia, iran, zambia and other countries, l...